In [1]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import uniform

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import  FunctionTransformer, OneHotEncoder, PolynomialFeatures, StandardScaler

In [2]:
# Enable diagrams to visualize pipelines
from sklearn import set_config
set_config(display="diagram")

In [3]:
def split_bmi_in_three(x: float) -> str:
    if x < 25:
        return "underweight_normal"
    if x < 30:
        return "overweight"
    return "obesity"

In [4]:
def tqdm_callback(desc, total):
    pbar = tqdm(total=total, desc=desc)

    def update(*args):
        pbar.update()
        time.sleep(0.1)  # Small sleep to see the progress increment

    return update

# Data Loading & Separating Features / Target

In [5]:
df = pd.read_csv("csvs/cleaned_dataset.csv")

In [6]:
y = df.pop("charges")
X = df

### Modifying `y`'s shape

In [7]:
y = np.log(y + 1)

# Preprocessing

## With Binning `bmi` Inside PipeLine

### Hold-Out

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    shuffle=True,
                                                    train_size=0.85,
                                                    random_state=42,
                                                    stratify=X['smoker'])

### Pipeline

In [9]:
X.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region'], dtype='object')

In [10]:
bmi_categorizer = FunctionTransformer(split_bmi_in_three)
ohe_nom = OneHotEncoder(drop="first", handle_unknown="ignore")
ohe_bin = OneHotEncoder(drop="if_binary", handle_unknown="ignore")
poly = PolynomialFeatures(degree=2)
std = StandardScaler()

In [11]:
en = ElasticNet(random_state=42, 
                max_iter=10_000, tol=1e-3
)

In [12]:
pipe_bmi = make_pipeline(bmi_categorizer, ohe_nom)
pipe_bmi

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function split_bmi_in_three at 0x7f62242a19e0>)),
                ('onehotencoder',
                 OneHotEncoder(drop='first', handle_unknown='ignore'))])

In [13]:
encoding = ColumnTransformer([
    ("bmi", pipe_bmi, ["bmi"]),
    ("bin", ohe_bin, ["sex", "smoker"]),
    ("ohe", ohe_nom, ["region"])
], remainder="passthrough")
encoding

ColumnTransformer(remainder='passthrough',
                  transformers=[('bmi',
                                 Pipeline(steps=[('functiontransformer',
                                                  FunctionTransformer(func=<function split_bmi_in_three at 0x7f62242a19e0>)),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['bmi']),
                                ('bin',
                                 OneHotEncoder(drop='if_binary',
                                               handle_unknown='ignore'),
                                 ['sex', 'smoker']),
                                ('ohe',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore'),
                                 ['region'])])

In [14]:
model = make_pipeline(encoding, poly, std, en)
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('bmi',
                                                  Pipeline(steps=[('functiontransformer',
                                                                   FunctionTransformer(func=<function split_bmi_in_three at 0x7f62242a19e0>)),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['bmi']),
                                                 ('bin',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['sex', 'smoker']),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['region'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('elasticnet',
                 ElasticNet(max_iter=10000, random_state=42, tol=0.001))])

## With Binning `bmi` Outside Pipeline

In [15]:
X_bmi_nom = X.copy()

In [16]:
X_bmi_nom.bmi = X_bmi_nom.bmi.apply(split_bmi_in_three)

### Hold-Out

In [17]:
X_bmi_nom_train, X_bmi_nom_test, y_train, y_test =\
train_test_split(X_bmi_nom, y,
                 shuffle=True,
                 train_size=0.85,
                 random_state=42,
                 stratify=X['smoker'])

### Pipeline

In [18]:
encoder = ColumnTransformer(
    transformers=[
        ("bin", ohe_bin, ["sex", "smoker"]),
        ("nom", ohe_nom, ["bmi", "region"])
    ],
    remainder="passthrough"
)
encoder

ColumnTransformer(remainder='passthrough',
                  transformers=[('bin',
                                 OneHotEncoder(drop='if_binary',
                                               handle_unknown='ignore'),
                                 ['sex', 'smoker']),
                                ('nom',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore'),
                                 ['bmi', 'region'])])

In [19]:
model = make_pipeline(encoder, poly, std, en)
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('bin',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['sex', 'smoker']),
                                                 ('nom',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['bmi', 'region'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('elasticnet',
                 ElasticNet(max_iter=10000, random_state=42, tol=0.001))])

# Training

In [24]:
%%time

params = {
    "elasticnet__alpha": uniform(0, 2),
    "elasticnet__l1_ratio": uniform(0, 1)
}

random_search = RandomizedSearchCV(
    model,
    param_distributions=params,
    n_iter=2_000,
    cv=10,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_bmi_nom_train, y_train)

/home/greg/projects/PolicyPriceAI/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.718e+00, tolerance: 8.441e-01
  model = cd_fast.enet_coordinate_descent(
/home/greg/projects/PolicyPriceAI/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.980e+00, tolerance: 8.600e-01
  model = cd_fast.enet_coordinate_descent(
/home/greg/projects/PolicyPriceAI/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the featu

CPU times: user 24.9 s, sys: 1.61 s, total: 26.5 s
Wall time: 32.8 s


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('bin',
                                                                               OneHotEncoder(drop='if_binary',
                                                                                             handle_unknown='ignore'),
                                                                               ['sex',
                                                                                'smoker']),
                                                                              ('nom',
                                                                               OneHotEncoder(drop='first',
                                                                                             handle_unknown='ignore'),
                                                                               ['bmi',
                                                                                'region'])])),
                                             ('polynomialfeatures',
                                              PolynomialFeatures()),
                                             ('standardscaler',
                                              StandardScaler()),
                                             ('elasticnet',
                                              ElasticNet(max_iter=10000,
                                                         random_state=42,
                                                         tol=0.001))]),
                   n_iter=2000, n_jobs=-1,
                   param_distributions={'elasticnet__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f62242603d0>,
                                        'elasticnet__l1_ratio': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f6225d23e90>},
                   random_state=42)

In [25]:
best_model = random_search.best_estimator_
best_model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('bin',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['sex', 'smoker']),
                                                 ('nom',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['bmi', 'region'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('elasticnet',
                 ElasticNet(alpha=0.00026938600897064013,
                            l1_ratio=0.5111291392372298, max_iter=10000,
                            random_state=42, tol=0.001))])

In [26]:
best_model.fit(X_bmi_nom_train, y_train)
best_model.score(X_bmi_nom_test, y_test)

0.9177335980741432

# 💿 Save model

In [27]:
joblib.dump(best_model, "model.joblib")

['model.joblib']